In [3]:
import mysql.connector

In [12]:
# Connect to server
cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="admin123")

# Get a cursor
cur = cnx.cursor()


In [13]:
cur.execute("show databases")
for x in cur:
    print(x)


('classicmodels',)
('information_schema',)
('mysql',)
('online_sales',)
('performance_schema',)
('sys',)
('yash',)
('yash1',)


## Get Basic Sales Summary from a Tiny MYSQL Database using Python

In [40]:
import mysql.connector
from mysql.connector import Error
from datetime import date, timedelta

def create_database_and_sample_data():
    """Create database and populate with sample sales data"""
    try:
        # Connect to MySQL server (without specifying a database)
        connection = mysql.connector.connect(
            host='localhost',
            user='root',  # Replace with your MySQL username
            password='admin123'   # Replace with your MySQL password
        )
        
        cursor = connection.cursor()
        
        # Create database
        cursor.execute("DROP DATABASE IF EXISTS tiny_sales")
        cursor.execute("CREATE DATABASE tiny_sales")
        cursor.execute("USE tiny_sales")
        
        # Create sales table
        cursor.execute("""
        CREATE TABLE sales (
            sale_id INT AUTO_INCREMENT PRIMARY KEY,
            sale_date DATE NOT NULL,
            customer_name VARCHAR(50),
            product_name VARCHAR(50) NOT NULL,
            category VARCHAR(30) NOT NULL,
            quantity INT NOT NULL,
            unit_price DECIMAL(10,2) NOT NULL,
            amount DECIMAL(10,2) NOT NULL
        )
        """)
        
        # Generate sample sales data
        products = [
            ('Wireless Headphones', 'Electronics', 59.99),
            ('Bluetooth Speaker', 'Electronics', 89.99),
            ('Smart Watch', 'Electronics', 199.99),
            ('T-Shirt', 'Clothing', 19.99),
            ('Jeans', 'Clothing', 39.99),
            ('Coffee Maker', 'Home', 49.99),
            ('Desk Lamp', 'Home', 29.99)
        ]
        
        customers = ['John Smith', 'Emma Johnson', 'Michael Brown', 'Sarah Davis', 'David Wilson']
        
        sales_data = []
        for i in range(50):  # Generate 50 random sales
            product = products[i % len(products)]
            quantity = (i % 3) + 1  # Random quantity 1-3
            unit_price = product[2]
            amount = round(quantity * unit_price, 2)
            
            sale_date = date.today() - timedelta(days=50 - i)
            customer = customers[i % len(customers)]
            
            sales_data.append((
                sale_date,
                customer,
                product[0],
                product[1],
                quantity,
                unit_price,
                amount
            ))
        
        # Insert sample data
        insert_query = """
        INSERT INTO sales (sale_date, customer_name, product_name, category, quantity, unit_price, amount)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        cursor.executemany(insert_query, sales_data)
        connection.commit()
        print(f"Created database with {len(sales_data)} sample sales records")
        
    except Error as e:
        print("Database error:", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def generate_sales_report():
    """Generate and display sales summary report"""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='tiny_sales',
            user='root',  # Replace with your MySQL username
            password='admin123'    # Replace with your MySQL password
        )
        
        cursor = connection.cursor(dictionary=True)
        
        print("\n" + "="*40)
        print("SALES SUMMARY REPORT".center(40))
        print("="*40)
        
        # 1. Overall Summary
        cursor.execute("""
            SELECT 
                COUNT(*) as total_transactions,
                SUM(quantity) as total_units_sold,
                SUM(amount) as total_revenue,
                AVG(amount) as avg_sale_amount,
                MIN(sale_date) as first_sale,
                MAX(sale_date) as last_sale
            FROM sales
        """)
        summary = cursor.fetchone()
        
        print("\nOVERALL PERFORMANCE:")
        print(f"- Period: {summary['first_sale']} to {summary['last_sale']}")
        print(f"- Total Transactions: {summary['total_transactions']}")
        print(f"- Units Sold: {summary['total_units_sold']}")
        print(f"- Total Revenue: ${summary['total_revenue']:,.2f}")
        print(f"- Average Sale: ${summary['avg_sale_amount']:,.2f}")
        
        # 2. Sales by Category
        cursor.execute("""
            SELECT 
                category,
                COUNT(*) as transactions,
                SUM(amount) as revenue,
                ROUND(SUM(amount) / (SELECT SUM(amount) FROM sales) * 100, 1) as percentage
            FROM sales
            GROUP BY category
            ORDER BY revenue DESC
        """)
        
        print("\nSALES BY CATEGORY:")
        for row in cursor:
            print(f"- {row['category']}: ${row['revenue']:,.2f} ({row['percentage']}%)")
        
        # 3. Top Products
        cursor.execute("""
            SELECT 
                product_name,
                SUM(quantity) as units_sold,
                SUM(amount) as revenue
            FROM sales
            GROUP BY product_name
            ORDER BY revenue DESC
            LIMIT 5
        """)
        
        print("\nTOP SELLING PRODUCTS:")
        for i, row in enumerate(cursor, 1):
            print(f"{i}. {row['product_name']}: {row['units_sold']} units (${row['revenue']:,.2f})")
        
        # 4. Recent Transactions
        cursor.execute("""
            SELECT 
                sale_date,
                customer_name,
                product_name,
                amount
            FROM sales
            ORDER BY sale_date DESC
            LIMIT 5
        """)
        
        print("\nRECENT TRANSACTIONS:")
        for row in cursor:
            print(f"- {row['sale_date']}: {row['customer_name']} bought {row['product_name']} (${row['amount']:,.2f})")
        
    except Error as e:
        print("Error generating report:", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def cleanup():
    """Remove the database (optional)"""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',  # Replace with your MySQL username
            password=''   # Replace with your MySQL password
        )
        
        cursor = connection.cursor()
        cursor.execute("DROP DATABASE IF EXISTS tiny_sales")
        print("\nDatabase cleaned up")
        
    except Error as e:
        print("Cleanup error:", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# Main execution
if __name__ == "__main__":
    # Step 1: Create database with sample data
    create_database_and_sample_data()
    
    # Step 2: Generate sales report
    generate_sales_report()

Created database with 50 sample sales records

          SALES SUMMARY REPORT          

OVERALL PERFORMANCE:
- Period: 2025-03-12 to 2025-04-30
- Total Transactions: 50
- Units Sold: 99
- Total Revenue: $7,119.01
- Average Sale: $142.38

SALES BY CATEGORY:
- Electronics: $5,159.56 (72.5%)
- Home: $1,139.72 (16.0%)
- Clothing: $819.73 (11.5%)

TOP SELLING PRODUCTS:
1. Smart Watch: 15 units ($2,999.85)
2. Bluetooth Speaker: 14 units ($1,259.86)
3. Wireless Headphones: 15 units ($899.85)
4. Coffee Maker: 15 units ($749.85)
5. Jeans: 14 units ($559.86)

RECENT TRANSACTIONS:
- 2025-04-30: David Wilson bought Wireless Headphones ($119.98)
- 2025-04-29: Sarah Davis bought Desk Lamp ($29.99)
- 2025-04-28: Michael Brown bought Coffee Maker ($149.97)
- 2025-04-27: Emma Johnson bought Jeans ($79.98)
- 2025-04-26: John Smith bought T-Shirt ($19.99)
